<a href="https://colab.research.google.com/github/Oluwatobi02/Fashionmodel/blob/main/FashionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers pillow pandas


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation, ViltProcessor, ViltForQuestionAnswering
from PIL import Image
import numpy as np
import pandas as pd
import torch.nn as nn
import threading

In [3]:
processor = SegformerImageProcessor.from_pretrained("sayeed99/segformer-b3-fashion")
model = AutoModelForSemanticSegmentation.from_pretrained("sayeed99/segformer-b3-fashion")
processor_nlp = ViltProcessor.from_pretrained("yanka9/vilt_finetuned_deepfashionVQA_v2")
model_nlp = ViltForQuestionAnswering.from_pretrained("yanka9/vilt_finetuned_deepfashionVQA_v2")

In [4]:
categories = model.config.id2label
del categories[0]
categories[1] = "shirt or blouse"
categories[2] = "top, t-shirt or sweatshirt"

In [5]:
other_cat = ["color", "style"]

In [6]:
def create_columns():
    columns = []
    for idx in range(1, 9):
        if idx in [52,53]:
            continue
        it_col = []
        it_col.append(f"item {idx} type")
        for i in other_cat:
            it_col.append(f"item {idx} {i}")
        columns.extend(it_col)
    return columns
    


In [7]:
fash_col = create_columns()

In [8]:
# with open("dataset.csv", "a") as file:
#     file.write(",".join(fash_col)+"\n")


In [9]:
def get_metadata(path, item):
    image = Image.open(path)
    answer = []
    questions = [f"what is the color of the {item}?", f"is this {item} casual or formal"]

    for text in questions:

    # prepare inputs
        encoding = processor_nlp(image, text, return_tensors="pt")

        # forward pass
        outputs = model_nlp(**encoding)
        logits = outputs.logits
        idx = logits.argmax(-1).item()
        answer.append(model_nlp.config.id2label[idx])
    return tuple(answer)

In [10]:
def ask_question(path, items):
    image = Image.open(path)
    text = f"{items}?"

    # prepare inputs
    encoding = processor_nlp(image, text, return_tensors="pt")

    # forward pass
    outputs = model_nlp(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    return model_nlp.config.id2label[idx]
    

In [29]:
takeouts = ["glasses", "collar", "sleeve", "buckle", "belt","pocket" ]

def get_categories(path):
    image = Image.open(path)
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits.cpu()
    unsampled_logits = nn.functional.interpolate(
        logits,
        size=image.size[::-1],
        mode="bilinear",
        align_corners=False
    )
    pred_seg = unsampled_logits.argmax(dim=1)[0]
    unique_cat = np.unique(pred_seg.cpu().numpy())
    result = []
    for group_no in unique_cat:
        if group_no in [1,2]:
            group = categories[group_no]
            result.append(ask_question(path, group))
        elif group_no == 0: continue
        else:
            result.append(categories[group_no])
    for i in takeouts:
        if i in result:
            result.remove(i)
    return result[:7]

            


In [19]:
def write_to_csv(row, csv_file="dataset.csv"):
    with open(csv_file, "a") as file:
        file.write(row + "\n")

In [20]:

def handle_image(path):
    labels = get_categories(path)
    result = []
    for label in labels:
        result.append(label)
        metadata = get_metadata(path, label)
        result.extend(metadata)
    t1 = threading.Thread(target=write_to_csv, args=(",".join(result),))
    t1.start()



In [21]:
images = [f"{i}.jpg" for i in range(1, 21)]

In [22]:
def main():
    print("running main...")
    for image_path in images:
        p1 = threading.Thread(target=handle_image, args=(f"./data/{image_path}",))
        p1.start()


In [23]:
main()

running main...


In [24]:
df = pd.read_csv("dataset.csv")
df

,item 1 type,item 1 color,item 1 style,item 2 type,item 2 color,item 2 style,item 3 type,item 3 color,item 3 style,item 4 type,...,item 5 style,item 6 type,item 6 color,item 6 style,item 7 type,item 7 color,item 7 style,item 8 type,item 8 color,item 8 style
0,dress,black and white,casual,pants,white,formal,shoe,black and white,formal,pocket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,jacket,brown,casual,pants,brown,casual,shoe,white,casual,hood,...,casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jacket,gray,casual,jacket,gray,casual,pants,black,casual,shoe,...,casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sweater,brown,formal,pants,white,formal,coat,brown,formal,shoe,...,formal,neckline,white,formal,NaN,NaN,NaN,NaN,NaN,NaN
4,jacket,gray,formal,cardigan,gray,formal,pants,gray,formal,shoe,...,brown,formal,neckline,white,formal,NaN,NaN,NaN,NaN,NaN
5,jacket,gray,casual,jacket,gray,casual,pants,black,formal,hat,...,casual,lapel,gray,formal,neckline,white,formal,NaN,NaN,NaN
6,sweater,tan,casual,cardigan,tan,casual,jacket,tan,casual,pants,...,casual,pocket,tan,casual,neckline,pink,formal,NaN,NaN,NaN
7,sweater,gray,casual,sweater,gray,casual,pants,tan,casual,sock,...,casual,pocket,gray,casual,neckline,blue,casual,NaN,NaN,NaN
8,sweater,blue,casual,jacket,blue,casual,pants,gray,casual,shorts,...,casual,shoe,gray,casual,bag,wallet,blue,casual,NaN,NaN
9,jacket,orange,casual,jacket,orange,casual,pants,tan,casual,shoe,...,orange,casual,lapel,orange,formal,pocket,orange,casual,NaN,NaN


In [28]:
# for col in df.columns:
#     if df[col].dtype == object:# convert
#         df[col] = df[col].astype('category')
#         df[col] = df[col].cat.codes

In [27]:
# df